In [1]:
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm


import seaborn as sns
import platform

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

# Loading and Understanding Data

In [2]:
df =pd.read_csv('C:/Users/raehy/Downloads/petition.csv', parse_dates=['start','end'])

In [3]:
df.shape

(377756, 8)

In [4]:
df.head()

,article_id,start,end,answered,votes,category,title,content
0,21,2017-08-19,2017-11-17,0,9,안전/환경,스텔라 데이지호에 대한 제안입니다.,스텔라 데이지호에 대한 제안입니다.\n3월31일 스텔라 데이지호가 침몰하고 5달째가...
1,22,2017-08-19,2017-11-17,0,17,기타,비리제보처를 만들어주세요.,현 정부에 국민들이 가장 원하는 것은 부패척결입니다. 우리 사회에 각종 비리들이 ...
2,23,2017-08-19,2017-09-03,0,0,미래,제2의 개성공단,"만일 하시는 대통령님 및 각 부처 장관님,주무관님들 안녕하세요!!\n전남 목포에서 ..."
3,24,2017-08-19,2017-08-26,0,53,일자리,공공기관 무조건적인 정규직전환을 반대합니다.,현정부에서 정규직 일자리를 늘리는 것에 찬성합니다. 그런데 공공기관 비정규직들은 인...
4,25,2017-08-19,2017-09-03,0,0,미래,제2의 개성공단,"만일 하시는 대통령님 및 각 부처 장관님,주무관님들 안녕하세요!!\n전남 목포에서 ..."


In [5]:
# Find out any missing data using isnull. Fortunately, there are only 1 missing data.

In [6]:
df.isnull().sum()

article_id    0
start         0
end           0
answered      0
votes         0
category      0
title         0
content       1
dtype: int64

In [7]:
# Find out the data type of features

In [8]:
df.dtypes

article_id             int64
start         datetime64[ns]
end           datetime64[ns]
answered               int64
votes                  int64
category              object
title                 object
content               object
dtype: object

In [9]:
# This is classification dataset in which the column "answered" is target/label, categorized in binary numeric form: 0 for "not answered" and 1 for "answered"

In [10]:
df.columns

Index(['article_id', 'start', 'end', 'answered', 'votes', 'category', 'title',
       'content'],
      dtype='object')

In [11]:
# here we can find that very small number of petitions were answered at 0.00077% due to the fact that only petitions that receive over 200,000 votes will be nominated for answer
df.describe()

,article_id,answered,votes
count,377756.000000,377756.000000,3.777560e+05
mean,239703.455924,0.000077,1.514069e+02
std,146382.863480,0.008761,4.842551e+03
min,21.000000,0.000000,0.000000e+00
25%,108933.750000,0.000000,1.000000e+00
50%,237637.000000,0.000000,5.000000e+00
75%,367937.250000,0.000000,1.500000e+01
max,492043.000000,1.000000,1.192049e+06


In [12]:
# We find out that only 77 petitions received over 200,000 votes; As noted in introduction, only petitions with over 200,000 votes can be nominated for being answered by the Presidential Office 
df_200000 = df.loc[df.votes > 200000]
df_200000.head(3)

,article_id,start,end,answered,votes,category,title,content
1752,1785,2017-09-03,2017-11-02,1,296330,인권/성평등,청소년이란 이유로 보호법을 악용하는 잔인무도한 청소년들이 늘어나고있습니다. 반드시 ...,안녕하십니까. 청소년보호법이란 명목하에 나쁜짓을 일삼는 청소년들이 너무나 많아지고 ...
10894,10949,2017-09-06,2017-12-05,1,615354,미래,조두순 출소반대,제발 조두순 재심다시해서 무기징역으로 해야됩니다!!!
18111,18278,2017-09-30,2017-10-30,1,235372,인권/성평등,낙태죄 폐지와 자연유산 유도약(미프진) 합법화 및 도입을 부탁드립니다.,안녕하세요. 존경하는 대통령님 의원님\n낙태죄 폐지를 청원합니다.\n현재 대한민국은...


In [13]:
# Let us find out the distribution of category/theme of petitions over 200,000 votes.
# 인권/성평등 (human right/gender equality) category nominated 23 petitions, followed by 안전/환경 (safety/environmental protection), nominated 10 petitions 

In [14]:
df_200000.category.value_counts()

인권/성평등         23
안전/환경          10
정치개혁            7
문화/예술/체육/언론     7
기타              7
외교/통일/국방        4
반려동물            4
육아/교육           3
보건복지            3
교통/건축/국토        3
경제민주화           2
성장동력            2
미래              1
행정              1
Name: category, dtype: int64

In [15]:
# only 29 petitions were answered, although 77 petitions received over 200,000 votes.
# This demonstrates that the Korean presidential office acts ambiguous in fulfilling the promise, and feels a lack of confidence in certain issues.
df.loc[df['answered']==1].count()

article_id    29
start         29
end           29
answered      29
votes         29
category      29
title         29
content       29
dtype: int64

In [1]:
# Since the objective of this study is based on prediction on the probability of being "nominated" for the answer, let's create a new column which categorizes petitions either nominable or not.


### Qualified petitions for the Presidential Office's answer
- petitions that receive over 200,000 votes must be answered by the Presidential Office

In [17]:
# Create a new column "answer" (not to be confused with the existing column "answered") to identify those petitions that received over 200,000 votes.
# These petitions should have been answered by the Presidential Office.
df['answer']= (df['votes'] >200000) == 1

In [18]:
# Let us find out petitions that received the most votes, It was an infamous murder incident in an internet cafe. 
answered_df = df.loc[df['answer']==1]
answered_df.sort_values('votes', ascending=False)

,article_id,start,end,answered,votes,category,title,content,answer
313314,408609,2018-10-17,2018-11-16,0,1192049,안전/환경,강서구 피시방 살인 사건. 또 심신미약 피의자입니다.,2018년 10월 14일 엊그제 일어난 강서구 피시방 살인사건에 대한 청원입니다.\...,True
208597,269548,2018-06-13,2018-07-13,0,714875,외교/통일/국방,"제주도 불법 난민 신청 문제에 따른 난민법, 무사증 입국, 난민신청허가 폐지/개헌 ...",2012년 난민법 제정으로 인해 외국인은 한달 무비자로 입국할 수 있으나 난민신청자...,True
10894,10949,2017-09-06,2017-12-05,1,615354,미래,조두순 출소반대,제발 조두순 재심다시해서 무기징역으로 해야됩니다!!!,True
118970,142600,2018-02-19,2018-03-21,1,614127,문화/예술/체육/언론,"김보름, 박지우 선수의 자격박탈과 적폐 빙상연맹의 엄중 처벌을 청원합니다","오늘 여자 단체전 팀추월에서 김보름, 박지우 선수는 팀전인데도 불구하고 개인의 영달...",True
183791,230552,2018-05-11,2018-06-10,1,419006,인권/성평등,여성도 대한민국 국민입니다. 성별 관계없는 국가의 보호를 요청합니다.,최근 홍대 누드크로키 모델의 불법촬영 사건이 있었습니다.\n사건은 굉장히 빠르게 처...,True
327873,426834,2018-10-31,2018-11-30,0,416093,인권/성평등,"132cm, 31kg의 왜소한 50대 여성이 180cm가 넘는 건장한 20세 남성에...",http://m.news1.kr/articles/?3464603\n관련 기사입니다....,True
242969,314584,2018-07-22,2018-08-21,0,413924,육아/교육,23개월 아기가 폭행에 장이 끊어져 죽었습니다.,고생하시는 청와대 관계자 및 대통령님.\n이 글을 꼭 한번만 읽어주세요.\n너무나 ...,True
303793,395955,2018-10-02,2018-11-01,1,406655,교통/건축/국토,음주운전 교통사고로 친구 인생이 박살났습니다. 제발 도와주세요,"9월 25일 새벽 2시 25분 경, 부산 해운대구 미포 오거리에서 술에 만취한 운전...",True
336559,437737,2018-11-09,2018-12-09,0,397079,인권/성평등,억울하게 떠나신 저의 아버지의 원한을 풀어주세요,46일동안 눈 한번 뜨시지 못한 채 11월 6일 새벽 4시에 저의 아버지가 억울하게...,True
340046,442093,2018-11-14,2018-12-14,0,365418,인권/성평등,이수역 폭행사건,11월 13일 새벽4시 경 이수역의 한 맥주집에서 남자 다섯명이 여성 두 명을 폭행...,True


### Category with the highest number of petitions 

In [19]:

category = pd.DataFrame(df['category'].value_counts()).reset_index()
category.columns = ['category', 'counts']
category

,category,counts
0,정치개혁,59020
1,기타,46449
2,인권/성평등,33738
3,안전/환경,29196
4,교통/건축/국토,26910
5,외교/통일/국방,25705
6,육아/교육,24861
7,보건복지,23608
8,일자리,22111
9,행정,19373


### Dates with the highest number of petitions 

In [20]:
# Which dates people send the most petitions?

In [21]:

start_df = pd.DataFrame(df['start'].value_counts()).reset_index()
start_df.columns = ['start', 'counts']
start_df = start_df.sort_values('counts', ascending=False)
print('Total Captured Period: {} days'.format(start_df.shape[0]))
start_df.head()

Total Captured Period: 509 days


,start,counts
0,2017-11-11,9623
1,2017-09-05,5952
2,2018-01-11,3368
3,2018-02-06,2631
4,2017-11-09,2487


### Category with the highest number of votes 

In [2]:
# Did all petitions received over 200,000 votes were answered? 
# Let's find out by category

In [23]:
# Generally, across categories shows a very low rate of being "answered" by the Presidential Office.
# For instance, petitions in 인권/성평등 (Human Right/Gender Equality ) category garnered over 12 million votes, nominated 23 petitions, but only 9 cases were answered.
petitions_unique = pd.pivot_table(df, index=['category'], aggfunc=np.sum)
petitions_best = petitions_unique.sort_values(by='votes', \
                                              ascending=False).reset_index()
petitions_best

,category,answer,answered,article_id,votes
0,인권/성평등,23.0,9,7329787166,12225998
1,안전/환경,10.0,2,6806887655,6512799
2,정치개혁,7.0,2,12787866627,5686172
3,기타,7.0,1,11097814909,4720310
4,육아/교육,3.0,1,5482023289,4420589
5,보건복지,3.0,2,5965078695,4269399
6,문화/예술/체육/언론,7.0,4,3948940469,4086382
7,외교/통일/국방,4.0,1,6404855753,3637926
8,교통/건축/국토,3.0,2,7909816091,2834727
9,반려동물,4.0,0,918432206,2023905


### Dates with the highest number of votes 

In [24]:
# Find out which dates received the highest number of votes. Autumn in 2018 seems a politically polemic time of Korea
petitions_start = pd.pivot_table(df, index=['start'], aggfunc=np.sum)
votes_df = petitions_start.sort_values(by='votes', ascending=False)
votes_df.loc[petitions_start['votes'] > 400000]

,answer,answered,article_id,votes
start,,,,
2018-10-17,1.0,0,378900198,1300523
2018-10-31,2.0,0,430964099,827096
2018-06-13,1.0,0,146187973,786157
2018-10-18,2.0,0,423557789,721524
2018-02-19,1.0,1,99332898,701520
2018-11-09,2.0,0,349439392,672819
2018-07-22,1.0,0,282639463,672491
2017-09-06,1.0,1,22268570,648209
2018-06-24,2.0,0,359345089,628925


###  Most Petition Sent (Count) vs Most Vote Casted (Votes)

In [25]:
# Merge two tables : 1) Dates by the highest petitions 2) Dates by the highest vote 

In [26]:
# There isn't a strong linear relation between counts and votes, which implies that a group of individuals flood with votes for specific cases.
votes_df = votes_df.reset_index()
hottest_day_df = start_df.merge(votes_df, on='start', how='left')
hottest_day_df.sort_values('counts', ascending=False).head()

,start,counts,answer,answered,article_id,votes
0,2017-11-11,9623,0.0,0,348559310,85074
1,2017-09-05,5952,0.0,0,38414241,48808
2,2018-01-11,3368,0.0,0,291069195,44570
3,2018-02-06,2631,0.0,0,342371897,83038
4,2017-11-09,2487,0.0,0,73459579,34774


###  Petitions grouped by unit of Time

In [27]:
# Count of votes by time unit : month, week, day and hour

In [28]:
df['start_month'] = df['start'].dt.month
df['start_dow'] = df['start'].dt.dayofweek
df['start_day'] = df['start'].dt.day
df['start_hour'] = df['start'].dt.hour

df.shape

(377756, 13)

In [29]:
# Autumn and Winter are usually politically polemic season.
df['start_month'].value_counts()

11    52847
9     40888
12    38498
1     37410
10    31687
8     28851
2     25857
7     25714
5     25499
6     25483
4     23227
3     21795
Name: start_month, dtype: int64

# Grouping data based on common theme using key words

In [30]:
# Let's inquire how many some hot topics were listed in petitions

In [31]:
# Cryptocurrency (가상화폐) was the hottest social topic in Korea.
# whopping 7911 petitions were related or contained cryptoptocurry content
crypto = df[( df.title.str.find('가상화폐') != -1 ) | ( df.content.str.find('가상화폐') != -1  )]
crypto.shape

(7911, 13)

In [3]:
# For a more advanced technique, I use ReGex module to identify the topic of petitions where I typed  keywords to see how many the words appeared in petition title and content

In [33]:
# For parental care, I picked key words - parental care, infant education, elementary school, nursey
import re
p = r'.*(돌봄|육아|초등|보육).*'
care = df[df['title'].str.match(p) |
           df['content'].str.match(p, flags=re.MULTILINE)]
care.shape

(14960, 13)

In [35]:
# For North Korea, I picked North Korea, Kim Jong Un, Reunification, Korean Peninsula and Missile
nk = r'.*(북한|김정은|통일|한반도|미사일).*'
northkorea= df[df['title'].str.match(nk) |
           df['content'].str.match(p, flags=re.MULTILINE)]
northkorea.shape

(20691, 13)

In [83]:
# For finance, I picked P2P,Bank, Finance, Stock, Stock Market,Short Sell
p = r'.*(P2P|은행|금융|주식|증권|공매도).*'
finance = df[df['title'].str.match(p) |
           df['content'].str.match(p, flags=re.MULTILINE)]
finance.shape

(19866, 13)


# Tokenization
### by using Soynlp library - Korean language tokenizer


In [37]:

pip install JPype1-0.6.3-cp36-none-win_amd64.whl

Note: you may need to restart the kernel to use updated packages.


ERROR: JPype1-0.6.3-cp36-none-win_amd64.whl is not a supported wheel on this platform.


In [38]:
pip install soynlp

Note: you may need to restart the kernel to use updated packages.


In [40]:
pip show soynlp


Name: soynlp
Version: 0.0.493
Summary: Unsupervised Korean Natural Language Processing Toolkits
Home-page: https://github.com/lovit/soynlp
Author: Lovit
Author-email: soy.lovit@gmail.com
License: UNKNOWN
Location: c:\users\raehy\anaconda3\lib\site-packages
Requires: psutil, scikit-learn, numpy, scipy
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [128]:
from soynlp.tokenizer import RegexTokenizer
import re

In [42]:
tokenizer = RegexTokenizer()
tokenizer

In [84]:
sample_index = 192190

In [85]:
sample_title = finance['title'][sample_index]
sample_title

'2009년도 탄생 적폐제도(?) 신속(조기)집행 폐지를 간절히 청원합니다.'

In [132]:
sample_content = finance['content'][sample_index]
sample_content[:1000]

'2009년도 탄생 적폐제도(?) 신속(조기)집행 폐지를 간절히 청원합니다.\\n국민의 일원으로써, 일선지자체의 공무원으로써,\\n2009년도부터 실시되어 현재까지 시행중인 적폐 제도, 신속(조기,균형) 집행의 폐지를 청원합니다.\\n2008년 리먼사태로 세계 금융위기가 발생하였다는 이유로 2009년도에 시작된 조기집행...\\n(지극히 제 개인적인 생각입니다다만. 지나고 나서 돌아보니, 솔직히 그것 때문에 시작했다고 생각이 들지 않습니다.)\\n어찌되었든, 현재까지 10년동안 그 제도같지 않은 제도가 이름만 달리하면서(조기→균형→신속) 시행되고 있는 것으로 보아,\\n우리 국가는 매년을 급박한 경제위기속에서 신속(조기)집행으로 버티어 내고 있는 중인 것으로 보입니다.\\n또한, 해당 제도 시행의 근거로 제시되는 「상반기 어려운 고용상황 및 거시경제 불안요인 등에 대응」에 따르면, 우리 국가는 금방이라도 부도가 날 지경인 것 같습니다.\\n현재 우리 대한민국은 정말 그러한 상황인가 되묻고 싶습니다.\\n지난 10년동안 각종 폐해와 문제점이 지적되고 있음에도, 도대체 어떤 이유로 현재까지 시행이 되고 있는지 이유를 찾을 수가 없습니다.\\n각종 언론매체 등을 통해 제기된 부작용을 나열하면,\\n“단기간 과다한 공사발주로 부실 설계 우려”,\\n“지자체 이자 수입 감소와 재정 압박 초래”,\\n“발주 후 설계변경이나 중복 집행 등의 예산낭비”,\\n“공사가 단기간에 집중되면서 공사현장에서 인건비나 자재비가 급등”,\\n“획일적 조기집행의 추진은 지방재정 안정성을 저해”,\\n“중앙정부의 예산조기집행은 지방자치와 지방재정 건전성 문제에 위배”\\n등의 문제점이 전문가, 지자체장, 지방의원, 공무원 등에 의해 지속적으로 제기되어 왔습니다.\\n또한 행정안전부에서도 이미 그 폐해, 문제점을 인지하고 있습니다.\\n-2018년도 행정안전부 지방재정 신속집행 추진계획 3페이지\\n○자치단체, 지방의회, 공무원 단체 등에서 신속집행에 대한 문제점* 등을 들어 지속적으로 개선

## Text Preprocessing
Prior to tokenization, sentences must be cleaned from non-word characters

In [144]:
#In preprocessing, I deleted \(newline), special characters, number and any letters or characters that aren't English or Korean. Also, Reduce multiple spaces to one 
def preprocessing(text):
    text = re.sub('\\\\n', ' ', text)
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    return text

In [89]:
sample_content = preprocessing(sample_content)
sample_content[:1000]

'    년도 탄생 적폐제도    신속 조기 집행 폐지를 간절히 청원합니다  국민의 일원으로써  일선지자체의 공무원으로써      년도부터 실시되어 현재까지 시행중인 적폐 제도  신속 조기 균형  집행의 폐지를 청원합니다      년 리먼사태로 세계 금융위기가 발생하였다는 이유로     년도에 시작된 조기집행     지극히 제 개인적인 생각입니다다만  지나고 나서 돌아보니  솔직히 그것 때문에 시작했다고 생각이 들지 않습니다   어찌되었든  현재까지   년동안 그 제도같지 않은 제도가 이름만 달리하면서 조기 균형 신속  시행되고 있는 것으로 보아  우리 국가는 매년을 급박한 경제위기속에서 신속 조기 집행으로 버티어 내고 있는 중인 것으로 보입니다  또한  해당 제도 시행의 근거로 제시되는  상반기 어려운 고용상황 및 거시경제 불안요인 등에 대응 에 따르면  우리 국가는 금방이라도 부도가 날 지경인 것 같습니다  현재 우리 대한민국은 정말 그러한 상황인가 되묻고 싶습니다  지난   년동안 각종 폐해와 문제점이 지적되고 있음에도  도대체 어떤 이유로 현재까지 시행이 되고 있는지 이유를 찾을 수가 없습니다  각종 언론매체 등을 통해 제기된 부작용을 나열하면   단기간 과다한 공사발주로 부실 설계 우려    지자체 이자 수입 감소와 재정 압박 초래    발주 후 설계변경이나 중복 집행 등의 예산낭비    공사가 단기간에 집중되면서 공사현장에서 인건비나 자재비가 급등    획일적 조기집행의 추진은 지방재정 안정성을 저해    중앙정부의 예산조기집행은 지방자치와 지방재정 건전성 문제에 위배  등의 문제점이 전문가  지자체장  지방의원  공무원 등에 의해 지속적으로 제기되어 왔습니다  또한 행정안전부에서도 이미 그 폐해  문제점을 인지하고 있습니다       년도 행정안전부 지방재정 신속집행 추진계획  페이지  자치단체  지방의회  공무원 단체 등에서 신속집행에 대한 문제점  등을 들어 지속적으로 개선 요구  이자수입 대폭 감소  상반기 공사집중으로 부실시공 우려  공사자

In [91]:
sentences = finance['content'].apply(preprocessing)

In [92]:
tokened_title = tokenizer.tokenize(sample_title)
tokened_title

['2009',
 '년도',
 '탄생',
 '적폐제도',
 '(?)',
 '신속',
 '(',
 '조기',
 ')',
 '집행',
 '폐지를',
 '간절히',
 '청원합니다',
 '.']

In [93]:
# Post - preprocessing tokenization
tokened_content = tokenizer.tokenize(sample_content)
tokened_content[:10]

['년도', '탄생', '적폐제도', '신속', '조기', '집행', '폐지를', '간절히', '청원합니다', '국민의']

In [68]:
# Compare with tokenization at the bottom

In [94]:
%time tokens = sentences.apply(tokenizer.tokenize)
tokens[:3]

Wall time: 1min 22s


7     [존경하옵는, 문재인대통령님께, 저는, 중국원양자원이라는, KOSPI, 상장사의, ...
11    [존경하는, 대통령님께, 코스피, 주식종목, 중국, 원양자원은, 현재, 계속되는, ...
54    [지장물관련, 국토부의, 해석, 기존, 건축물의, 철거, 공사에, 관한, 사항의, ...
Name: content, dtype: object

In [95]:
tokens[sample_index][:10]

['년도', '탄생', '적폐제도', '신속', '조기', '집행', '폐지를', '간절히', '청원합니다', '국민의']

# Word2Vec


Word2Vec(Word Embedding to Vector) vectorizes words in a way that estimate distance of "similarity" between words.
Unlinke other vectorization techniques, Word2Vec "embeds" meaning of words along its relationship with other words or within a sentence. 

In [77]:
!conda install -y gensim

Solving environment: ...working... failed with current_repodata.json, will retry with next repodata source.
Initial quick solve with frozen env failed.  Unfreezing env and trying again.
Solving environment: ...working... failed with current_repodata.json, will retry with next repodata source.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\raehy\Anaconda3

  added / updated specs:
    - gensim


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    boto3-1.10.39              |             py_0          89 KB
    botocore-1.13.39           |             py_0         2.8 MB
    bz2file-0.98               |           py37_1          14 KB
    conda-4.7.12               |           py37_0         3.0 MB
    gensim-3.4.0               |   py37hfa6e2cd_0        17.4 MB
    jmespath-0.9.4             |             py_0          22 KB
    s3transfer-0.2.1 

In [96]:

import logging
logging.basicConfig(
    format='%(asctime)s : %(levelname)s : %(message)s', 
    level=logging.INFO)

In [103]:
from gensim.models import Word2Vec
model = Word2Vec(tokens, size=100, window=5, min_count=5, workers=4, sg=0)



2019-12-27 20:15:21,065 : INFO : collecting all words and their counts
2019-12-27 20:15:21,068 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-12-27 20:15:22,903 : INFO : PROGRESS: at sentence #10000, processed 2204292 words, keeping 321082 word types
2019-12-27 20:15:25,215 : INFO : collected 578625 word types from a corpus of 4574205 raw words and 19866 sentences
2019-12-27 20:15:25,217 : INFO : Loading a fresh vocabulary
2019-12-27 20:15:26,170 : INFO : effective_min_count=5 retains 92560 unique words (15% of original 578625, drops 486065)
2019-12-27 20:15:26,172 : INFO : effective_min_count=5 leaves 3868526 word corpus (84% of original 4574205, drops 705679)
2019-12-27 20:15:27,187 : INFO : deleting the raw counts dictionary of 578625 items
2019-12-27 20:15:27,212 : INFO : sample=0.001 downsamples 7 most-common words
2019-12-27 20:15:27,218 : INFO : downsampling leaves estimated 3833095 word corpus (99.1% of prior 3868526)
2019-12-27 20:15:27,995 : I

In [118]:
# total word count in dictionary
len(model.wv.vocab)

92560

In [119]:
# Retrieve the top 10 words in the dictionary.  Most of the words are stopwords
vocab = model.wv.vocab
sorted(vocab, key=vocab.get, reverse=True)[:30]


['수',
 '있습니다',
 '있는',
 '그',
 '이',
 '년',
 '합니다',
 '하는',
 '더',
 '하고',
 '대한',
 '및',
 '할',
 '월',
 '제',
 '그리고',
 '한',
 '등',
 '없는',
 '저는',
 '이런',
 '것입니다',
 '있다',
 '일',
 '왜',
 '많은',
 '위해',
 '것은',
 '것이',
 '같은']

# Word Similarity

In [112]:
# Let's find similar words for 북한 (North Korea)
# Result words are : Kim Jong Un, Socialism, Approval Rating, Trump, Neighbour Country, Destroyed etc 
model_result2 = model.wv.most_similar("북한")
print(model_result2)

[('김정은', 0.8172783851623535), ('사회주의', 0.8027055263519287), ('지지율', 0.801215648651123), ('트럼프', 0.7984379529953003), ('존경받는', 0.7860324382781982), ('국가주도형', 0.7811722159385681), ('성공한', 0.7759419679641724), ('좌파', 0.7736547589302063), ('망한다는거', 0.7735177278518677), ('이웃나라', 0.7713704109191895)]


In [115]:
# Let's find similar words for 비트코인 (Bitcoin)
# Result words are : Cryptocurrency, Coin, Market, China, Global etc 
model_result3 = model.wv.most_similar("비트코인")
print(model_result3)

[('암호화폐', 0.8115838766098022), ('코인', 0.7905029058456421), ('글로벌', 0.7833611965179443), ('가상화폐', 0.7778764963150024), ('시장은', 0.7748391628265381), ('바이오', 0.7748149037361145), ('코스피', 0.7631715536117554), ('중국', 0.7573140263557434), ('비트코인을', 0.7555472254753113), ('전세계', 0.7493956089019775)]


In [124]:
# Similarity between "North Korea" and "War"
model.wv.similarity('북한','전쟁')

0.6011367

In [125]:
# Similarity between "USA" and "War"
model.wv.similarity('미국','전쟁')

0.6575829

In [126]:
# Similarity between "Kim Jong Un" and "Trump"
model.wv.similarity('김정은','트럼프')

0.8453425

In [127]:
# Similiarity between "Cryptocurrency" and "Regulation"
model.wv.similarity('가상화폐','규제')

0.71403813

# Stopwords

-  From the word2vec analysis, we found out that the most commonly used words were insignificant words or stopwards.
-  There isn't stopword library for Korean language, so I have to manually assign a bag of stopwords. Fortunately, there is Korean stopword "dictionary" https://www.ranks.nl/stopwords/korean

In [129]:
# First, start with preprocessing, deleting \(newline), special characters, and any letters or characters that aren't English or Korean.
# Reduce multiple spaces to one 
def preprocessing(text):
    text = re.sub('\\\\n', ' ', text)
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    text = re.sub(' +', ' ', text)
    return text

In [139]:
# Second, removal of stopwords, I identified few common stopwords from contents.
def remove_stopwords(text):
    stops = ['수', '있는', '있습니다', '그', '년도', '에', '합니다', 
             '하는', '및', '제', '할', '하고', '더', '대한', '한', 
             '그리고', '월', '저는', '없는', '것입니다', '등', '일', 
             '많은', '이런', '것은', '왜', '같은', 
             '없습니다', '위해', '한다','따르면']
    # Eliminate stopwords
    meaningful_words = [w for w in text if not w in stops]
    return ''.join(meaningful_words)

In [142]:
# Let's bring up a sample petition to verify preprocessing 
sample_content = preprocessing(sample_content)
sample_content[:1000]

' 년도 탄생 적폐제도 신속 조기 집행 폐지를 간절히 청원합니다 국민의 일원으로써 일선지자체의 공무원으로써 년도부터 실시되어 현재까지 시행중인 적폐 제도 신속 조기 균형 집행의 폐지를 청원합니다 년 리먼사태로 세계 금융위기가 발생하였다는 이유로 년도에 시작된 조기집행 지극히 제 개인적인 생각입니다다만 지나고 나서 돌아보니 솔직히 그것 때문에 시작했다고 생각이 들지 않습니다 어찌되었든 현재까지 년동안 그 제도같지 않은 제도가 이름만 달리하면서 조기 균형 신속 시행되고 있는 것으로 보아 우리 국가는 매년을 급박한 경제위기속에서 신속 조기 집행으로 버티어 내고 있는 중인 것으로 보입니다 또한 해당 제도 시행의 근거로 제시되는 상반기 어려운 고용상황 및 거시경제 불안요인 등에 대응 에 따르면 우리 국가는 금방이라도 부도가 날 지경인 것 같습니다 현재 우리 대한민국은 정말 그러한 상황인가 되묻고 싶습니다 지난 년동안 각종 폐해와 문제점이 지적되고 있음에도 도대체 어떤 이유로 현재까지 시행이 되고 있는지 이유를 찾을 수가 없습니다 각종 언론매체 등을 통해 제기된 부작용을 나열하면 단기간 과다한 공사발주로 부실 설계 우려 지자체 이자 수입 감소와 재정 압박 초래 발주 후 설계변경이나 중복 집행 등의 예산낭비 공사가 단기간에 집중되면서 공사현장에서 인건비나 자재비가 급등 획일적 조기집행의 추진은 지방재정 안정성을 저해 중앙정부의 예산조기집행은 지방자치와 지방재정 건전성 문제에 위배 등의 문제점이 전문가 지자체장 지방의원 공무원 등에 의해 지속적으로 제기되어 왔습니다 또한 행정안전부에서도 이미 그 폐해 문제점을 인지하고 있습니다 년도 행정안전부 지방재정 신속집행 추진계획 페이지 자치단체 지방의회 공무원 단체 등에서 신속집행에 대한 문제점 등을 들어 지속적으로 개선 요구 이자수입 대폭 감소 상반기 공사집중으로 부실시공 우려 공사자재 품귀현상 발생 지역경제 활성화 파급효과 미미 무리한 실적경쟁으로 막대한 행정력 낭비 제도의 성과 분석과 평가 없이 매년 반복 시행 등 국회 년 국정